In [19]:
import numpy as np
import PIL.Image as Image
import glob
import cv2 
code2label = {
    "Bedroom":0,
    "Coast":1,
    "Forest":2,
    "Highway":3,
    "Industrial":4,
    "InsideCity":5,
    "Kitchen":6,
    "LivingRoom":7,
    "Mountain":8,
    "Office":9,
    "OpenCountry":10,
    "Store":11,
    "Street":12,
    "Suburb":13,
    "TallBuilding":14
}
def load_img2np(filepath):
    img = cv2.imread(filepath,cv2.IMREAD_GRAYSCALE)
    return cv2.resize(img, (16, 16))
def load_data(path):
    path_pattern = path + '/**/*.jpg'
    files_list = glob.glob(path_pattern, recursive=True)
    images = []
    labels = []
    for file_name in files_list:
        label_name = file_name.split('\\')[1]
        images.append(load_img2np(file_name))
        labels.append(code2label[label_name])
    return np.asarray(images), np.asarray(labels)
image_train, label_train = load_data('hw5_data/train')
image_test, label_test = load_data('hw5_data/test')

In [42]:
# K-NN
from collections import deque
def distance(img1,img2):
    return np.mean(np.square(img1-img2))
K = 25
acc = 0
for img_test, label in zip(image_test,label_test):
    nn = deque()
    for i, img_train in enumerate(image_train):
        dist = distance(img_test,img_train)
        if i == 0:
            nn.append((i,dist))
            continue
        for j in range(len(nn)):
            if dist < nn[j][1]:
                nn.insert(j,(i,dist))
                if len(nn) > K:
                    nn.pop()
                break
    count = np.zeros(15)
    for (idx,dist) in nn:
        count[label_train[idx]] += 1
    pred = count.argmax()
    acc += (label == pred)
print("acc: {:.2f}%".format(acc/label_test.shape[0]*100.))

acc: 20.67%
